<a href="https://colab.research.google.com/github/siddengineer/YOUTUBE-PDFS/blob/main/Temperature_ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Step 1: Create the dataset
data = {
    "Area": ["Kashmir", "Kashmir", "Kashmir", "Kashmir", "Kashmir",
             "Rajasthan", "Rajasthan", "Rajasthan", "Rajasthan", "Rajasthan",
             "Kashmir", "Kashmir", "Kashmir", "Rajasthan", "Rajasthan",
             "Kashmir", "Kashmir", "Rajasthan", "Rajasthan", "Kashmir"],
    "Temperature": [-8, -5, 0, 2, -3,
                    46, 42, 45, 47, 40,
                    -6, 3, -2, 43, 39,
                    -10, 1, 48, 38, -4],
    "Condition": ["Snowfall", "Snowfall", "Cloudy", "Sunny", "Fog",
                  "Sandstorm", "Sunny", "Cloudy", "Sunny", "Fog",
                  "Snowfall", "Sunny", "Fog", "Sandstorm", "Sunny",
                  "Snowfall", "Cloudy", "Sunny", "Cloudy", "Snowfall"],
    "WindSpeed": [12, 15, 8, 5, 10,
                  30, 20, 22, 35, 18,
                  14, 7, 9, 28, 19,
                  20, 6, 33, 17, 11],
    "SchoolTime": ["10:00 AM", "09:30 AM", "09:00 AM", "08:30 AM", "09:30 AM",
                   "07:30 AM", "07:00 AM", "07:00 AM", "07:30 AM", "08:00 AM",
                   "10:00 AM", "08:30 AM", "09:00 AM", "07:30 AM", "08:00 AM",
                   "10:00 AM", "09:00 AM", "07:30 AM", "08:00 AM", "10:00 AM"],
    "Mode": ["Online", "Online", "Offline", "Offline", "Online",
             "Online", "Offline", "Offline", "Online", "Offline",
             "Online", "Offline", "Online", "Online", "Offline",
             "Online", "Offline", "Online", "Offline", "Online"]
}

df = pd.DataFrame(data)

# Step 2: Encode categorical values
le_condition = LabelEncoder()
le_schooltime = LabelEncoder()
le_mode = LabelEncoder()

df["ConditionEncoded"] = le_condition.fit_transform(df["Condition"])
df["SchoolTimeEncoded"] = le_schooltime.fit_transform(df["SchoolTime"])
df["ModeEncoded"] = le_mode.fit_transform(df["Mode"])

# Step 3: Define features and targets
features = df[["Temperature", "ConditionEncoded", "WindSpeed"]]

target_time = df["SchoolTimeEncoded"]
target_mode = df["ModeEncoded"]

# Step 4: Train models
X_train_time, X_test_time, y_train_time, y_test_time = train_test_split(features, target_time, test_size=0.2, random_state=42)
X_train_mode, X_test_mode, y_train_mode, y_test_mode = train_test_split(features, target_mode, test_size=0.2, random_state=42)

time_model = RandomForestClassifier()
mode_model = RandomForestClassifier()

time_model.fit(X_train_time, y_train_time)
mode_model.fit(X_train_mode, y_train_mode)

# Step 5: Define the AI Agent
class SchoolAIAgent:
    def __init__(self, time_model, mode_model, le_condition, le_schooltime, le_mode):
        self.time_model = time_model
        self.mode_model = mode_model
        self.le_condition = le_condition
        self.le_schooltime = le_schooltime
        self.le_mode = le_mode

    def predict(self, temp, condition, windspeed):
        cond_encoded = self.le_condition.transform([condition])[0]

        # ✅ Pass input as DataFrame with correct column names
        input_data = pd.DataFrame([{
            'Temperature': temp,
            'ConditionEncoded': cond_encoded,
            'WindSpeed': windspeed
        }])

        time_encoded = self.time_model.predict(input_data)[0]
        mode_encoded = self.mode_model.predict(input_data)[0]

        school_time = self.le_schooltime.inverse_transform([time_encoded])[0]
        mode = self.le_mode.inverse_transform([mode_encoded])[0]

        print(f"\n📍 Based on the weather conditions:")
        print(f"   Temperature: {temp}°C")
        print(f"   Condition: {condition}")
        print(f"   Wind Speed: {windspeed} km/h")
        print(f"\n🕒 Suggested School Time: {school_time}")
        print(f"🧠 Recommended Mode: {'✅ Offline' if mode == 'Offline' else '💻 Online'}")

# Step 6: Use the agent
agent = SchoolAIAgent(time_model, mode_model, le_condition, le_schooltime, le_mode)

# 🌨️ Kashmir Example
agent.predict(temp=-7, condition="Snowfall", windspeed=14)

# ☀️ Rajasthan Example
agent.predict(temp=46, condition="Sunny", windspeed=22)


📍 Based on the weather conditions:
   Temperature: -7°C
   Condition: Snowfall
   Wind Speed: 14 km/h

🕒 Suggested School Time: 10:00 AM
🧠 Recommended Mode: 💻 Online

📍 Based on the weather conditions:
   Temperature: 46°C
   Condition: Sunny
   Wind Speed: 22 km/h

🕒 Suggested School Time: 07:00 AM
🧠 Recommended Mode: ✅ Offline
